# **Etiquetado para PLN con Rubrix**

## **¿Qué es Rubrix?**
### *Una herramienta gratuita y de código abierto para construir y mejorar datos de entrenamiento para PLN.*

Rubrix se compone de dos elementos:

- Una **aplicación web** para explorar, anotar y revisar datos.
- Una **librería Python** para crear datasets, pre-anotar, anotar programáticamente, mejorar datos, etc.

## **Cómo te podemos ayudar**

Si tienes dudas tenemos una comunidad muy activa y un equipo dedicado a Rubrix:

1. Únete a nuestro Slack (ver enlace en la web https://www.rubrix.ml/)
2. Escribe una issue en Github: https://github.com/recognai/rubrix

## **Si te gusta Rubrix, ¿cómo nos puedes ayudar?**

1. Agradecemos cualquier tipo de feedback, sugerencias, etc.
2. Para dar visibilidad al proyecto, dale una estrella en Github: https://github.com/recognai/rubrix
3. Comparte con tus contactos!
4. Si te apetece escribir sobre NLP práctico y datos, nuestro blog está abierto para cualquier persona.


## **Guión del taller**
0. Cómo instalar y lanzar Rubrix
1. Cómo subir o crear datasets para anotar para clasificación de texto
2. Cómo pre-anotar un dataset con un modelo pre-entrenado para clasificación de texto
3. Como etiquetar un dataset para clasificación de texto
4. Cómo entrenar un clasificador de texto
5. Cómo subir o crear datasets para anotar para NER
6. Cómo etiquetar un dataset para NER


## **Cómo instalar y lanzar Rubrix**

Necesitamos:

1. Elasticsearch (Recomendable lanzarlo usando Docker)
2. Python >= 3.7
3. Instalar rubrix con `pip install rubrix[server]`

Cómo lo lanzamos:

1. Arrancamos Elasticsearch (con esto tenemos la base de datos y el motor de búsqueda). 
2. Lanzamos la aplicación web de Rubrix con `python -m rubrix`
3. Recomendable usar Jupyter notebooks (JupyterLab, VS Code, etc.). Para usarlo con Colab necesitaremos tener una instancia Cloud de Rubrix (ver guía despliegue en AWS)

Para este taller necesitamos además:

In [ ]:
!pip install snorkel datasets transformers torch spacy -qqq

## **1. Cómo subir o crear un dataset para clasificación de texto**

Usando la librería python en Rubrix.


### Usando objetos `TextClassificationRecord`

In [2]:
import rubrix as rb
from rubrix import read_datasets

# usando las clases de registro de cada tarea: TextClassificationRecord, TokenClassificationRecord, etc.
rb.log(
    rb.TextClassificationRecord(text="mi primer registro"), 
    "test_dataset"
)

  0%|          | 0/1 [00:00<?, ?it/s]

1 records logged to http://localhost:6900/ws/rubrix/test_dataset


BulkResponse(dataset='test_dataset', processed=1, failed=0)

### Usando from_pandas

In [3]:
import pandas as pd

# usando pandas para leer un csv, json, etc.
df = pd.read_csv("https://raw.githubusercontent.com/recognai/pln_con_rubrix/main/datos/tweets_en_es.csv"); df

,Unnamed: 0,text,source
0,0,Se imaginan a los chicos agradeciendo por el p...,tweets_pos_clean.txt
1,1,"Eclesiastes4:9-12 ♡ Siempre, promesa :) https...",tweets_pos_clean.txt
2,2,"@pedroj_ramirez Qué saborío, PJ. ya no compart...",tweets_pos_clean.txt
3,3,Buenos dias para todos. Feliz inicio de semana...,tweets_pos_clean.txt
4,4,"@pepedom @bquintero Gracias! No es así, deja c...",tweets_pos_clean.txt
...,...,...,...
248305,70729,.@EsperanzAguirre y @mdcospedal se presentan a...,tweets_clean.txt
248306,70730,Desde aquí twiteo siempre http://t.co/JcH0yKr3\n,tweets_clean.txt
248307,70731,Es un honor y una necesidad política estar a p...,tweets_clean.txt
248308,70732,"""La salud y la vida de las mujeres vale más qu...",tweets_clean.txt


In [ ]:
rb.log(rb.DatasetForTextClassification().from_pandas(df[0:10]), "pandas_ds")

### Usando from_datasets y leyendo datasets del Hub de Hugging Face

In [ ]:
from datasets import load_dataset

dataset = load_dataset("muchocine", split="train") ; dataset

In [ ]:
rb_dataset = rb.DatasetForTextClassification().from_datasets(
    dataset, 
    inputs=['review_body', 'review_summary'], 
    annotation="star_rating"
)

rb.log(rb_dataset, "rb_muchocine")

## **2. Cómo pre-anotar un dataset**

In [ ]:
from transformers import pipeline

dataset = load_dataset("rubrix/muchocine_aspects", split="train")

In [23]:
nlp = pipeline(
    "zero-shot-classification", 
    model="Recognai/zeroshot_selectra_small", 
    return_all_scores=True,
)

Downloading:   0%|          | 0.00/997 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/337 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/378k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [49]:
nlp("La historia es lo mejor de la película", candidate_labels=labels, hypothesis_template=template)

{'sequence': 'La historia es lo mejor de la película',
 'labels': ['direccción', 'interpretacción', 'aspectos formales', 'guión'],
 'scores': [0.7319302558898926,
  0.18849410116672516,
  0.051197148859500885,
  0.028378522023558617]}

In [47]:
labels = ["guión", "direccción", "aspectos formales", "interpretacción"]
template = "La frase es sobre {}"

records = []

for record in dataset.select(range(10)):
    prediction = nlp(record['text'], labels)

    records.append(
        rb.TextClassificationRecord(
            text=record["text"],
            prediction=list(zip(prediction['labels'], prediction['scores'])),
        )
    )

rb.log(records, name="news_zeroshot")

  0%|          | 0/10 [00:00<?, ?it/s]

10 records logged to http://localhost:6900/ws/rubrix/news_zeroshot


BulkResponse(dataset='news_zeroshot', processed=10, failed=0)

## **3. Cómo etiquetar un dataset para clasificación de texto**

In [ ]:
dataset = load_dataset("rubrix/muchocine_aspects", split="train")

rb.log(read_datasets(dataset, task="TextClassification"),"muchocine_aspects")

## **4. Cómo entrenar un clasificador de texto**

In [132]:
ds = rb.load("muchocine_aspect", query="status:Validated", as_pandas=False).prepare_for_training()

### Fine-tune con transformers

In [133]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import numpy as np
from transformers import Trainer
from datasets import load_metric
from transformers import TrainingArguments

model_id = "Recognai/selectra_small"

# tokenize our datasets
tokenizer = AutoTokenizer.from_pretrained(model_id)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

ds = ds.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [135]:
# split the data into a training and evalutaion set
train_dataset, eval_dataset = ds.train_test_split(test_size=0.2, seed=42).values()

In [ ]:
id2label = {idx:label for idx,label in enumerate(ds.features["label"].names)}
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=len(ds.features["label"].names))

In [137]:
training_args = TrainingArguments(
    "muchocine_aspects",
    evaluation_strategy="epoch",
    logging_steps=30,
    num_train_epochs=2
)

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

### **5. Usando weak supervision**

In [55]:
from rubrix.labeling.text_classification import load_rules, WeakLabels

weak_labels = WeakLabels(dataset="muchocine_aspect_rules")
weak_labels.summary()

Preparing rules:   0%|          | 0/7 [00:00<?, ?it/s]

Applying rules:   0%|          | 0/69227 [00:00<?, ?it/s]

,label,coverage,annotated_coverage,overlaps,conflicts,correct,incorrect,precision
interpreta*,{interpretación},0.028269,0.127072,0.004102,0.003279,18,5,0.782609
trama,{guión},0.012322,0.116022,0.001603,0.001069,19,2,0.904762
banda sonora,{aspectos formales},0.006168,0.027624,0.001517,0.000968,4,1,0.800000
fotografía,{aspectos formales},0.008754,0.033149,0.002557,0.002008,4,2,0.666667
reparto,{interpretación},0.007280,0.038674,0.001647,0.000823,7,0,1.000000
(guion AND NOT actor*),{guión},0.020324,0.110497,0.003351,0.002817,12,8,0.600000
director*,{dirección},0.039074,0.171271,0.004117,0.004117,21,10,0.677419
total,"{guión, interpretación, aspectos formales, dir...",0.112427,0.558011,0.009129,0.007223,85,28,0.752212


In [ ]:
from rubrix.labeling.text_classification import Snorkel

# create the label model
label_model = Snorkel(weak_labels)

# fit the model
label_model.fit()

# test it with labeled test set
label_model.score()

In [94]:
# get your training records with the predictions of the label model
records_for_training = label_model.predict()

# optional: log the records to a new dataset in Rubrix
#rb.log(records_for_training, name="snorkel_results")

# extract training data
training_data = pd.DataFrame(
    [
        {"text": rec.text, "annotation": rec.prediction[0][0]}
        for rec in records_for_training
    ]
)

In [95]:
# quick look at our training data with the weak labels from our label model
with pd.option_context('display.max_colwidth', None):
    display(training_data)

,text,annotation
0,La trama de El nombre de la rosa es muy sencilla: en vísperas de que en una abadía benedictina se celebre una cumbre teológica algunos monjes aparecen siniestramente asesinados.,guión
1,"Las interacciones con el juez (el inolvidable Spencer Tracy), y las de éste con el personaje interpretado por Marlene Dietrich, conforman un producto de enorme calidad y cuyo extenso metraje, que llega a las tres horas, se pasa en un suspiro.",interpretación
2,"Por otra parte, la factura técnica está cuidada, tanto en maquillaje y FX como en fotografía, sonido y música, pero tan sólo brillan de veras esas ovejas amenazadoras.",aspectos formales
3,"Mientras, el tío, interpretado brillantemente por Steve Carell, ha dado el paso contrario, ha pasado de ser un triunfador a ser un perdedor.",interpretación
4,"El mejor ejemplo para resumir la frialdad de la película es el gran villano de la película, un tipo llamado Arcángel de Jesús Montoya, papel que interpreta (es un decir) Luis Tosar, recuperando el 'toque español' de 'Collateral'.",interpretación
...,...,...
7677,"En esta los rostros de los personajes, los ojos, transmiten emociones y a pesar de su diseño caricaturesco pueden llegar a parececernos seres reales, algo que nunca sucedía con la película interpretada por Tom Hanks.",interpretación
7678,"Aunque la secundaria Laura Linney tampoco ha de desmerecerse, pues su interpretación es, como de costumbre, excelente.",interpretación
7679,"Otros puntos altos son en primer término, la excelente fotografía de William C. Mellor, en un intenso blanco y negro y bello juego de contrastes, visible hasta en las fotos que acompañan el texto.",aspectos formales
7680,"Los sustos baratos tampoco ayudan mucho, a decir verdad, y el giro final que coge la trama se ve venir a leguas.",guión


### Push to hub para usarlo como dataset de entrenamiento (usando Colab, AutoNLP, etc.)

In [ ]:
rb.DatasetForTextClassification.from_pandas(
    training_data
).prepare_for_training(
).push_to_hub("rubrix/muchocine_aspectos", split="train")

In [ ]:
rb.load(
    "muchocine_aspect_rules", 
    query="status:Validated", 
    as_pandas=False
).prepare_for_training(
).push_to_hub("rubrix/muchocine_aspectos", split="validation")

### Entrenar un baseline en scikit-learn

In [88]:
# for the test set, we can retrieve the records with validated annotations 
df_test = rb.load("muchocine_aspect_rules", query="status:Validated")

# transform data to match our training set format
df_test['annotation'] = df_test['annotation'].apply(
    lambda r: label_model.weak_labels.label2int[r]
)

In [119]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# define our final classifier
classifier = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB())
])

# fit the classifier
classifier.fit(
    X=training_data.text.tolist(),
    y=training_data.annotation.values
)

Pipeline(steps=[('vect', CountVectorizer()), ('clf', MultinomialNB())])

In [125]:
classifier.predict(["Buena música"])

array(['aspectos formales'], dtype='<U17')

### Ejercicio: Usar Weasel para entrenar un transformers final directamente con reglas

Ver https://rubrix.readthedocs.io/en/master/guides/weak-supervision.html#Joint-Model-with-Weasel

## **6. Cómo subir o crear datasets para anotar para NER**

### Usando objetos TokenClassification

**IMPORTANTE**: El texto pasado como `text` tiene que enviarse tokenizado en el parámetro `tokens`. Para ello es recomendable usar `spaCy` o `tokenizers` e idealmente el mismo tokenizador que se va a usar en tiempo de inferencia.

In [141]:
record = rb.TokenClassificationRecord(
    text="Mi nombre es Daniel Vila",
    tokens="Mi nombre es Daniel Vila".split() # esto es un MAL tokenizador
)
rb.log(record, "ejemplo_ner")

  0%|          | 0/1 [00:00<?, ?it/s]

1 records logged to http://localhost:6900/ws/rubrix/ejemplo_ner


BulkResponse(dataset='ejemplo_ner', processed=1, failed=0)

### Usando from_pandas

In [142]:
# usando pandas para leer un csv, json, etc.
df = pd.read_csv("https://raw.githubusercontent.com/recognai/pln_con_rubrix/main/datos/tweets_en_es.csv"); df

,Unnamed: 0,text,source
0,0,Se imaginan a los chicos agradeciendo por el p...,tweets_pos_clean.txt
1,1,"Eclesiastes4:9-12 ♡ Siempre, promesa :) https...",tweets_pos_clean.txt
2,2,"@pedroj_ramirez Qué saborío, PJ. ya no compart...",tweets_pos_clean.txt
3,3,Buenos dias para todos. Feliz inicio de semana...,tweets_pos_clean.txt
4,4,"@pepedom @bquintero Gracias! No es así, deja c...",tweets_pos_clean.txt
...,...,...,...
248305,70729,.@EsperanzAguirre y @mdcospedal se presentan a...,tweets_clean.txt
248306,70730,Desde aquí twiteo siempre http://t.co/JcH0yKr3\n,tweets_clean.txt
248307,70731,Es un honor y una necesidad política estar a p...,tweets_clean.txt
248308,70732,"""La salud y la vida de las mujeres vale más qu...",tweets_clean.txt


In [150]:
import spacy

df = df[0:50]

nlp = spacy.load("es_core_news_md")
df['tokens'] = df['text'].apply(
    lambda r: [t.text for t in nlp(r)]
)

In [ ]:
rb.log(rb.DatasetForTokenClassification().from_pandas(df), "pandas_ds_ner")

### Usando el Hugging Face Hub

In [ ]:
rb_ner_dataset = read_datasets(load_dataset("rubrix/muchocine_ner", split="unlabelled"), task="TokenClassification")

In [ ]:
rb.log(rb_ner_dataset, "muchocine_ner")

## **7. Cómo etiquetar un dataset para NER**